In [10]:
import pandas as pd
#df = pd.read_csv('hkcensus2011/scripts/data/combined/census.csv')
# census-e.csv is superset of census.csv
# It incorporates natural language names besides the identifiers
df = pd.read_csv('hkcensus2011/scripts/data/combined/census-e.csv')

In [12]:
df.head()

,region,district,area,table,row,column,value,region_name,district_name,area_name,table_name,row_name,column_name
0,nt,p,p03,0,tab0_chinese,tab0_male,6884,New Territories,Tai Po,Chung Ting,Ethnicity,Chinese,Male
1,nt,p,p02,0,tab0_chinese,tab0_male,6497,New Territories,Tai Po,Tai Po Central,Ethnicity,Chinese,Male
2,nt,p,p01,0,tab0_chinese,tab0_male,7432,New Territories,Tai Po,Tai Po Hui,Ethnicity,Chinese,Male
3,nt,p,p07,0,tab0_chinese,tab0_male,6801,New Territories,Tai Po,Fu Ming Sun,Ethnicity,Chinese,Male
4,nt,p,p06,0,tab0_chinese,tab0_male,7912,New Territories,Tai Po,Yee Fu,Ethnicity,Chinese,Male


In [18]:
print(len(df))
area = df[df.area=='a01']
print(len(area))

171970
444


In [144]:
areas = df['area'].unique()
tables = df['table'].unique()

In [30]:
# Show the mapping of tables. 
columns = area[['table', 'table_name']]
columns.drop_duplicates()

,table,table_name
253,0,Ethnicity
6325,1,Usual Language
13741,2,Marital Status
21157,3,Educational Attainment
29809,4,Economic Activity Status
39697,5,Monthly Income from Main Employment (HK$)
55765,6,Household Composition
59061,7,Household Size
61533,8,Type of Housing
67713,9,Tenure of Accommodation


In [146]:
def identifier_to_cell(identifier):
    import re
    cell = identifier.split('_')[0]
    r = re.match(r'(\D+)(\d+)', cell)
    column = r.groups()[0]
    row = int(r.groups()[1])
    return (column, row)

def series_to_median(df):
    df['id_tuple'] = df['row'].apply(identifier_to_cell)
    df.sort('id_tuple')
    s = df['value'].sum()
    acc = 0.0 
    for i, row in df.iterrows():
        acc += row['value']
        if acc >= s / 2.0:
            # Return DataFrame rather than Series
            return pd.DataFrame([tuple(row.values)], columns=list(row.index), index=[i])
        
def series_to_modal(df):
    df['id_tuple'] = df['row'].apply(identifier_to_cell)
    df.sort('id_tuple')
    s = df['value'].sum()
    cur = -1
    for i, row in df.iterrows():
        if row['value'] > cur:
            cur = row['value']
            cur_df = pd.DataFrame([tuple(row.values)], columns=list(row.index), index=[i])
    return cur_df

STRATEGY = [
    # nane, table, column, 
    ('Median Age Group Male', 12, 'l6_male', series_to_median),
    ('Median Age Group Female', 12, 'm6_female', series_to_median),
    ('Median Age Group Both', 12, 'n6_both', series_to_median),
    ('Median Monthly Domestic Household Income Active', 18, 'l118_active', series_to_median),
    ('Median Monthly Domestic Household Income All', 18, 'n118_households', series_to_median),
    ('Median Monthly Domestic Household Mortgage Payment and Loan Repayment (HK$)', 19, 'n143_with', series_to_median),
    ('Median Monthly Domestic Household Rent (HK$)', 20, 'n162_they', series_to_median),
    ('Median Monthly Income from Main Employment (HK$) Male', 5, 'c77_male', series_to_median),
    ('Median Monthly Income from Main Employment (HK$) Female', 5, 'd77_female', series_to_median),
    ('Median Monthly Income from Main Employment (HK$) Both', 5, 'e77_both', series_to_median),
    ('Median Household Size', 7, 'e129_domestic', series_to_median),
    ('Modal Ethnicity Male', 0, 'tab0_male', series_to_modal),
    ('Modal Ethnicity Female', 0, 'tab0_female', series_to_modal),
    ('Modal Ethnicity Both', 0, 'tab0_both', series_to_modal),
    ('Modal Usual Language Male', 1, 'c18_male', series_to_modal),
    ('Modal Usual Language Female', 1, 'd18_female', series_to_modal),
    ('Modal Usual Language Both', 1, 'e18_both', series_to_modal),
    ('Modal Marital Status Male', 2, 'c28_male', series_to_modal),
    ('Modal Marital Status Female', 2, 'd28_female', series_to_modal),
    ('Modal Marital Status Both', 2, 'e28_both', series_to_modal),
    ('Educational Attainment Under 15', 3, 'c43_under', series_to_modal),
    ('Educational Attainment Over 15', 3, 'd43_and', series_to_modal),
    ('Educational Attainment Total', 3, 'e43_total', series_to_modal),
    ('Economic Activity Status Male', 4, 'c61_male', series_to_modal),
    ('Economic Activity Status Female', 4, 'd61_female', series_to_modal),
    ('Economic Activity Status Both', 4, 'e61_both', series_to_modal),
    # Stopped at the sub form "4. Household Characteristics"
]

profiles = []
for a in areas:
    area = df[df.area==a]
    area_profile = []
    for s in STRATEGY:
        #print(area[area.table==s[1]])
        filtered_data = area[(area.table==s[1])][(area.column==s[2])]
        re = s[3](filtered_data)
        #print(re[['area', 'area_name', 'row', 'row_name', 'value']])
        stat_str = re.iloc[0]['row_name']
        area_str = re.iloc[0]['area_name']
        area_profile.append(stat_str)
    profiles.append(tuple(area_profile))
    
#profiles

#df.sort('id_tuple')
#sorted(list(area[area.table==12][area.column=='l6_male'].row), key=identifier_to_cell)

/Users/hupili/Desktop/initium/hk_median/venv/lib/python3.4/site-packages/pandas/core/frame.py:1819: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  "DataFrame index.", UserWarning)


[('40 - 44',
  '40 - 44',
  '40 - 44',
  '20,000 - 24,999',
  '20,000 - 24,999',
  '4,000 - 5,999',
  '6,000 - 7,999',
  '10,000 - 14,999',
  '8,000 - 9,999',
  '10,000 - 14,999',
  '3',
  'Chinese',
  'Chinese',
  'Chinese',
  'Cantonese',
  'Cantonese',
  'Cantonese',
  'Married',
  'Married',
  'Married',
  'Primary',
  'Sixth Form',
  'Sixth Form',
  'Employee',
  'Employee',
  'Employee'),
 ('40 - 44',
  '40 - 44',
  '40 - 44',
  '25,000 - 29,999',
  '25,000 - 29,999',
  '6,000 - 7,999',
  '6,000 - 7,999',
  '15,000 - 19,999',
  '10,000 - 14,999',
  '10,000 - 14,999',
  '3',
  'Chinese',
  'Chinese',
  'Chinese',
  'Cantonese',
  'Cantonese',
  'Cantonese',
  'Married',
  'Married',
  'Married',
  'No schooling',
  'Sixth Form',
  'Sixth Form',
  'Employee',
  'Employee',
  'Employee'),
 ('45 - 49',
  '45 - 49',
  '45 - 49',
  '15,000 - 19,999',
  '10,000 - 14,999',
  '4,000 - 5,999',
  '2,000 - 3,999',
  '10,000 - 14,999',
  '8,000 - 9,999',
  '10,000 - 14,999',
  '2',
  'Chinese

In [152]:
headers = [s[0] for s in STRATEGY]
profile_df = pd.DataFrame(profiles, columns=headers, index=areas)
profile_df.to_csv('area_profiles.csv')

In [134]:
area[area.table==4]

,region,district,area,table,row,column,value,region_name,district_name,area_name,table_name,row_name,column_name
29809,hk,a,a01,4,a63_employees,c61_male,3785,Hong Kong Island,Central and Western,Chung Wan,Economic Activity Status,Employee,Male
30221,hk,a,a01,4,a64_employers,c61_male,325,Hong Kong Island,Central and Western,Chung Wan,Economic Activity Status,Employer,Male
30633,hk,a,a01,4,a65_self-employed,c61_male,360,Hong Kong Island,Central and Western,Chung Wan,Economic Activity Status,Self-employed,Male
31045,hk,a,a01,4,a66_unpaid,c61_male,3,Hong Kong Island,Central and Western,Chung Wan,Economic Activity Status,Unpaid family workers,Male
31457,hk,a,a01,4,a68_home-makers,c61_male,10,Hong Kong Island,Central and Western,Chung Wan,Economic Activity Status,Home-maker,Male
31869,hk,a,a01,4,a69_students,c61_male,808,Hong Kong Island,Central and Western,Chung Wan,Economic Activity Status,Student,Male
32281,hk,a,a01,4,a70_retired,c61_male,929,Hong Kong Island,Central and Western,Chung Wan,Economic Activity Status,Retired,Male
32693,hk,a,a01,4,a71_others,c61_male,597,Hong Kong Island,Central and Western,Chung Wan,Economic Activity Status,Others,Male
33105,hk,a,a01,4,a63_employees,d61_female,3539,Hong Kong Island,Central and Western,Chung Wan,Economic Activity Status,Employee,Female
33517,hk,a,a01,4,a64_employers,d61_female,165,Hong Kong Island,Central and Western,Chung Wan,Economic Activity Status,Employer,Female
